In [1]:
from dotenv import load_dotenv
load_dotenv()

True

#  Calling

## __ call __

In [8]:
from langchain import PromptTemplate, LLMChain, OpenAI
from langchain.chat_models import ChatOpenAI

prompt_template = "What is the word to replace the following: {word}"

llm = OpenAI(model_name='gpt-3.5-turbo', temperature=0)

llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))

llm_chain("artificial")

/Users/charanmannuru/miniconda3/envs/LLM/lib/python3.10/site-packages/langchain/llms/openai.py:179: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/charanmannuru/miniconda3/envs/LLM/lib/python3.10/site-packages/langchain/llms/openai.py:753: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


{'word': 'artificial', 'text': 'synthetic'}

## Apply

In [10]:
input_list = [
    {"word": "artificial"},
    {"word": "intelligence"},
    {"word": "robot"}
]

# llm_chain.apply(input_list) # batch processing only for some models.

## Generate

In [ ]:
llm_chain.generate(input_list)

## Predict

### Multiple

In [13]:
prompt_template = "Looking at the context of '{context}'. What is a approapriate word to replace the following; {word}?"

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=prompt_template, input_variables=['word', 'context'])
)

llm_chain.predict(word='fan', context='object')

'air conditioner'

In [14]:
llm_chain.predict(word="fan", context="humans")

'supporter'

### from String

In [15]:
template = """Looking at the context of '{context}'. What is a approapriate word to replace the following: {word}?"""
llm_chain = LLMChain.from_string(llm=llm, template=template)

In [16]:
llm_chain.predict(word="fan", context="object")

'air conditioner'

## Parsers

In [17]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template= """List all possible words as substitute for 'artificial' as comma separated."""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=[], output_parser=output_parser)
)

llm_chain.predict()

'synthetic, man-made, fake, simulated, imitation, faux, counterfeit, ersatz, fabricated, manufactured'

In [18]:
llm_chain.predict_and_parse()

/Users/charanmannuru/miniconda3/envs/LLM/lib/python3.10/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


['synthetic',
 'man-made',
 'fake',
 'simulated',
 'imitation',
 'faux',
 'ersatz',
 'fabricated',
 'counterfeit',
 'pseudo']

# Adding Memory Conversational Chain

In [27]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

template = """List all possible words as substitute for 'artificial' as comma separated.

Current conversation:
{history}

{input}"""

conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=["history", "input"], output_parser=output_parser),
    memory=ConversationBufferMemory()
)

conversation.predict_and_parse(input="Answer briefly. write the first 3 options.")


['synthetic', 'man-made', 'simulated']

In [28]:
conversation.predict_and_parse(input="And the next 4?")

['AI: fake', 'imitation', 'faux', 'counterfeit']

In [29]:
conversation.predict_and_parse(input="And the next 4?")

['AI: fabricated', 'ersatz', 'pseudo', 'unnatural']

In [30]:
conversation.predict_and_parse(input="How may have u given me until now")

['synthetic',
 'man-made',
 'simulated',
 'fake',
 'imitation',
 'faux',
 'counterfeit',
 'fabricated',
 'ersatz',
 'pseudo',
 'unnatural']

# Debug

In [32]:
conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=['history', 'input'], output_parser=output_parser),
    memory=ConversationBufferMemory(),
    verbose=True,
)

conversation.predict_and_parse(input="Answer briefly, write the first 3 options")

/Users/charanmannuru/miniconda3/envs/LLM/lib/python3.10/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




> Entering new  chain...
Prompt after formatting:
List all possible words as substitute for 'artificial' as comma separated.

Current conversation:


Answer briefly, write the first 3 options

> Finished chain.


['synthetic', 'man-made', 'simulated']

In [33]:
conversation.predict_and_parse(input="And the next 4?")



> Entering new  chain...
Prompt after formatting:
List all possible words as substitute for 'artificial' as comma separated.

Current conversation:
Human: Answer briefly, write the first 3 options
AI: synthetic, man-made, simulated

And the next 4?

> Finished chain.


['imitation', 'fake', 'faux', 'fabricated']

# Sequentail Chain

In [ ]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[llm_chain, conversation], verbose=True)

# Custom Chain

In [44]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain
from typing import Dict, List


class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self):
        # Union of the inpout keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)
    
    @property
    def output_keys(self):
        return ["concat_output"]
    
    def _call(self, inputs: Dict[str, str]):
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

In [ ]:
prompt_1 = PromptTemplate(input_variables=['word'],
                          template=" what is the meaning of the following word '{word}'?",
                          )
chain_3 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=['word'],
    template="What is a word to replace the following: {word}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_3, chain_2=chain_2)
concat_output = concat_chain.run('artificial')
print(f"Concatenated Output:\n{concat_output}")

Concatenated Output:
Artificial means made or produced by human beings rather than occurring naturally. It can also refer to something that is not genuine or real.Synthetic


In [42]:
print(f"Concatenated Output:\n{concat_output}")

Concatenated Output:
Artificial means made or produced by human beings rather than occurring naturally. It can also refer to something that is not genuine or real.Synthetic
